In [0]:
!rm -rf ./RL_2020
!git clone https://github.com/HanyangTechAI/RL_2020.git
!rm -rf ./RL_2020/.git

In [0]:
# 필요한 패키지를 설치해준다.
!cat ./RL_2020/lab02_deep_reinforcement_learning/requirements.txt
!echo '----------------------------------'
!pip install -r ./RL_2020/lab02_deep_reinforcement_learning/requirements.txt

## 정책을 근사하는 신경망
CartPole-v0은 observation으로 네 개의 값이 들어오고, 에이전트가 할 수 있는 행동은 두 개 이다. 따라서 우리가 만들 신경망은 입력이 네 개이고 출력이 두 개여야 한다.

In [0]:
# PyTorch를 사용하기 위한 모듈을 불러온다
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.distributions import Categorical

In [0]:
# 아래 ???을 채워봅시다.

class PolicyNet(nn.Module):
  def __init__(self):
    super(PolicyNet, self).__init__()

    self.fc1 = nn.Linear(4, 16)
    self.fc2 = nn.Linear(16, 2)

    self.act = nn.Sigmoid()

  def forward(self, x):
    x = self.act(self.fc1(x))

    return F.softmax(self.fc2(x), dim=0)

In [0]:
import random
import numpy as np

# REINFORCE를 이용한 에이전트를 만들자.
class Agent:
  def __init__(self, actions):
    # 할 수 있는 행동의 집합
    self.actions = actions

    # 학습률로, 한 번에 얼마나 학습할지 결정한다.
    self.learning_rate = 0.01

    # 감가율
    self.discount_factor = 0.9

    # 정책을 근사할 신경망이다.
    self.p_net = PolicyNet()

    # 신경망을 최적화 할 optimizer
    self.opt = optim.Adam(self.p_net.parameters(), lr=self.learning_rate)

    # 학습 데이터를 저장할 리스트
    self.log_probs = []
    self.rewards = []

  # state를 받아 할 행동을 구하는 메소드
  def get_action(self, state):
    policy = self.p_net(state)

    m = Categorical(policy)
    action = m.sample()

    self.log_probs.append(m.log_prob(action))

    return action.item()

  # 보상을 바탕으로 Q Network를 업데이트 하는 메소드
  def train(self):
    R = 0
    policy_loss = []
    returns = []

    for r in self.rewards[::-1]:
      R = r + self.discount_factor * R
      returns.insert(0, R)

    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + 1e-5)

    for log_prob, R in zip(self.log_probs, returns):
      policy_loss.append(-log_prob * R)

    self.opt.zero_grad()

    loss = sum(policy_loss)
    loss.backward()

    self.opt.step()

    self.log_probs.clear()
    self.rewards.clear()

In [0]:
import gym

# 위에서 만든 에이전트를 바탕으로 학습을 시켜보자.
env = gym.make('CartPole-v0')
agent = Agent(env.action_space)

# reward를 저장할 리스트
rewards = []

# 5000 에피소드를 수행한다.
for episode in range(1, 5000 + 1):
  state = env.reset()
  
  # state를 PyTorch의 Tensor형으로 변환해야한다.
  state = torch.FloatTensor(state)

  total_reward = 0

  while True:
    action = agent.get_action(state)
    next_state, reward, done, _ = env.step(action)

    agent.rewards.append(reward)
    total_reward += reward

    # next_state도 Tensor형으로 변환해야한다.
    next_state = torch.FloatTensor(next_state)

    if done:
      break

    state = next_state

  rewards.append(total_reward)
  agent.train()

  # 이전 50개 에피소드의 평균 reward가 190보다 크거나 같으면 학습 중단
  if np.mean(rewards[-min(50, len(rewards)):]) >= 190:
    print('Early stopped (after {} episodes)'.format(episode))
    break

In [0]:
# 학습이 진행됨에 따른 reward 변화를 그래프로 그린다.
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(range(1, len(rewards) + 1), rewards)
plt.show()